# Get own implied volatilities and compare them to Deribits

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
pd.set_option('display.max_columns', 500)
import seaborn as sns
import plotly.graph_objects as go
import datetime
from scipy import stats as sps

from utils import get_human_timestamp, get_difference_between_now_and_expirety_date
import plotly
import plotly.graph_objs as go
from typing import Union
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
df = pd.read_csv(f"./datasets/deribit_options_chain_2022-12-01_OPTIONS.csv")
data = df.copy()

In [27]:
# Ньютон-Рафсон
def get_implied_volatility_call(
    option_type: Union["call", "put"],
    C: float,
    K: float,
    T: float,
    F: float,
    r: float = 0.0,
    error: float = 0.001,
) -> float:
    """
    Function to count implied volatility via given params of option, using Newton-Raphson method :

    Args:
        C (float): Option market price(USD).
        K (float): Strike(USD).
        T (float): Time to expiration in years.
        F (float): Underlying price.
        r (float): Risk-free rate.
        error (float): Given threshhold of error.

    Returns:
        float: Implied volatility in percent.
    """
    vol = 1.0
    dv = error + 1
    while abs(dv) > error:
        d1 = (np.log(F / K) + 0.5 * vol**2 * T) / (vol * np.sqrt(T))
        d2 = d1 - vol * np.sqrt(T)
        D = np.exp(-r * T)
        if option_type.lower() == "call":
            price = F * sps.norm.cdf(d1) - K * sps.norm.cdf(d2) * D
        elif option_type.lower() == "put":
            price = -F * sps.norm.cdf(-d1) + K * sps.norm.cdf(-d2) * D
        else:
            raise ValueError("Wrong option type, must be 'call' or 'put' ")
        Vega = F * np.sqrt(T / np.pi / 2) * np.exp(-0.5 * d1**2)
        PriceError = price - C
        dv = PriceError / Vega
        vol = vol - dv
    # in percents, as deribit shows
    return vol * 100

In [26]:
test = data.sample(100)
for index, row in test.iterrows():
    iv = get_implied_volatility_call(
        option_type = row["type"], 
        C = row["mark_price"]*row["underlying_price"],
        K = row["strike_price"],
        T = get_difference_between_now_and_expirety_date(row["expiration"], row["timestamp"]),
        F = row["underlying_price"],
        r = 0.0,
        error = 0.001
    )
    deribit_value = row["mark_iv"]
    print(f"""Deribit: {deribit_value}, counted: {iv}, error: {2*(deribit_value - iv)/(iv + deribit_value) * 100}% """)

Deribit: 71.37, counted: 71.39695195223145, error: -0.03775657021866015% 
Deribit: 74.02, counted: 73.98776239411238, error: 0.04356204751177066% 
Deribit: 52.89, counted: 53.64914230033624, error: -1.4250955732235902% 
Deribit: 121.33, counted: 130.6526608863188, error: -7.399446337718217% 
Deribit: 82.9, counted: 82.97295238687349, error: -0.08796176329379034% 
Deribit: 54.55, counted: 54.5599660670842, error: -0.01826793178190972% 
Deribit: 78.29, counted: 78.23918884426692, error: 0.0649222756576591% 
Deribit: 87.81, counted: 87.9116375538704, error: -0.11568018063711166% 
Deribit: 66.21, counted: 66.19224571969087, error: 0.026818699656662306% 
Deribit: 65.33, counted: 65.29210253685616, error: 0.05802611106055692% 
Deribit: 58.69, counted: 58.70861126089382, error: -0.031706100598516675% 
Deribit: 68.74, counted: 68.76788955274758, error: -0.040564294657264376% 
Deribit: 64.12, counted: 64.16951314482733, error: -0.07718969947517679% 
Deribit: 91.91, counted: 91.92371632206915, e